In [1]:
from model_builder_pme import model_builder
from data_gen import simulate_private_equity_cashflows
import numpy as np
from pme_calc import sp500_data, xirr, ln_pme, ks_pme, pme_plus, direct_alpha, moic, index_weighted_cashflows

In [2]:
cashflows = simulate_private_equity_cashflows()

In [3]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

In [4]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [5]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, cashflows)
model

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 1.756740915858248,
  'Market KS-PME': 1.756740915858248,
  'Timing Alpha KS-PME': 2.220446049250313e-16,
  'Strategy Alpha KS-PME': 2.220446049250313e-16,
  'Geography Alpha KS-PME': 2.220446049250313e-16,
  'Sizing Alpha KS-PME': 0.0,
  'Residual Alpha KS-PME': -6.661338147750939e-16},
 {'Portfolio Direct Alpha': 0.030067494557816248,
  'Market Direct Alpha': 0.030067494557816248,
  'Timing Direct Alpha': -2.1510571102112408e-16,
  'Strategy Direct Alpha': -2.1510571102112408e-16,
  'Geography Direct Alpha': -2.1510571102112408e-16,
  'Sizing Direct Alpha': 0.0,
  'Residual Direct Alpha': 6.453171330633722e-16})

### Monte-Carlo simulation

In [6]:
number_of_simulations = 50
fund_size = 200
ks_pme_results = []
da_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    ks_pme_results.append(model[0])
    da_results.append(model[1])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [8]:
avg_ks_pme_decomposition = average_dicts(ks_pme_results)
avg_da_decomposition = average_dicts(da_results)

In [9]:
avg_ks_pme_decomposition

{'Portfolio KS-PME': 1.6902159462716029,
 'Market KS-PME': 1.7567409158582463,
 'Timing Alpha KS-PME': -0.0257868028290213,
 'Strategy Alpha KS-PME': 0.0010376063718946814,
 'Geography Alpha KS-PME': -0.010738874217418077,
 'Sizing Alpha KS-PME': 0.011473612662661474,
 'Residual Alpha KS-PME': -0.04251051157476189}

In [10]:
avg_da_decomposition

{'Portfolio Direct Alpha': 0.0299970909487243,
 'Market Direct Alpha': 0.030067494557816262,
 'Timing Direct Alpha': 0.0004955177813772588,
 'Strategy Direct Alpha': 7.019505562856667e-05,
 'Geography Direct Alpha': -0.00034419620795437264,
 'Sizing Direct Alpha': 0.00042308821162907186,
 'Residual Direct Alpha': -0.0007150084497724773}

## Individual effects

### Strategy effect

In [19]:
venture_ids = cashflows[cashflows['Strategy'] == 'Venture']['FundID'].unique()
buyout_ids = cashflows[cashflows['Strategy'] == 'Buyout']['FundID'].unique()
venture_ids = np.append(venture_ids,buyout_ids[0]) # we need at least one venture fund to make the model work
buyout_ids = np.append(buyout_ids,venture_ids[0]) # we need at least one buyout fund to make the model work

In [20]:
def portfolio_cashflows_by_id(cashflows, ids):

    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in ids}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(ids)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [21]:
port_cfs_venture = portfolio_cashflows_by_id(cashflows, venture_ids)
port_cfs_venture = port_cfs_venture.copy()
port_cfs_buyout = portfolio_cashflows_by_id(cashflows, buyout_ids)
port_cfs_buyout = port_cfs_buyout.copy()

In [22]:
model_venture = model_builder(cashflows, port_cfs_venture)
model_venture

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 2.401761648096196,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.04516107838518413,
  'Strategy Alpha KS-PME': 0.5950075340889445,
  'Geography Alpha KS-PME': 0.002585480689198283,
  'Sizing Alpha KS-PME': -0.0851666244071545,
  'Residual Alpha KS-PME': 0.03160525187251517},
 {'Portfolio Direct Alpha': 0.04899488425679962,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -0.0009896510646039483,
  'Strategy Direct Alpha': 0.017620553410394035,
  'Geography Direct Alpha': 0.00010989055028954448,
  'Sizing Direct Alpha': -0.0019517863795702195,
  'Residual Direct Alpha': 0.0002098025615779181})

In [23]:
model_buyout = model_builder(cashflows, port_cfs_buyout)
model_buyout

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 1.3513309036043886,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.020701016953176055,
  'Strategy Alpha KS-PME': -0.572883483338007,
  'Geography Alpha KS-PME': -0.020661770697454784,
  'Sizing Alpha KS-PME': 0.012594358189074395,
  'Residual Alpha KS-PME': 0.05009173216607543},
 {'Portfolio Direct Alpha': 0.015593400056193878,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -1.2165429978686082e-05,
  'Strategy Direct Alpha': -0.019297876746297134,
  'Geography Direct Alpha': -0.0006912312260666242,
  'Sizing Direct Alpha': 0.0006232818671272032,
  'Residual Direct Alpha': 0.0009753164126968285})

### Geo effect

In [24]:
america_ids = cashflows[cashflows['Geography'] == 'North America']['FundID'].unique()
row_ids = cashflows[cashflows['Geography'] == 'Rest of World']['FundID'].unique()
america_ids = np.append(america_ids,row_ids[0]) # we need at least one venture fund to make the model work
row_ids = np.append(row_ids,america_ids[0]) # we need at least one buyout fund to make the model work

In [25]:
port_cfs_america = portfolio_cashflows_by_id(cashflows, america_ids)
port_cfs_america = port_cfs_america.copy()
port_cfs_row = portfolio_cashflows_by_id(cashflows, row_ids)
port_cfs_row = port_cfs_row.copy()

In [26]:
model_america = model_builder(cashflows, port_cfs_america)
model_america

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 2.9234900418333742,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.01010251631009118,
  'Strategy Alpha KS-PME': 0.01555662600379315,
  'Geography Alpha KS-PME': 1.0149748833779615,
  'Sizing Alpha KS-PME': 0.043962980773027294,
  'Residual Alpha KS-PME': -0.04379301624919307},
 {'Portfolio Direct Alpha': 0.06030327802602387,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -0.0007174535298718407,
  'Strategy Direct Alpha': 0.0005309317671245845,
  'Geography Direct Alpha': 0.02683560180705686,
  'Sizing Direct Alpha': 0.0005106185027175039,
  'Residual Direct Alpha': -0.0008524956997155234})

In [27]:
model_row = model_builder(cashflows, port_cfs_row)
model_row

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 0.953038584576485,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.039220106234034935,
  'Strategy Alpha KS-PME': -0.0014452120885930508,
  'Geography Alpha KS-PME': -0.9504942973170445,
  'Sizing Alpha KS-PME': -0.02737631606032276,
  'Residual Alpha KS-PME': 0.06868343203860383},
 {'Portfolio Direct Alpha': -0.0025423950351025773,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -0.0005100760736808582,
  'Strategy Direct Alpha': -0.0001163272224877776,
  'Geography Direct Alpha': -0.03659088864308122,
  'Sizing Direct Alpha': -0.0014847792469657604,
  'Residual Direct Alpha': 0.002163600972400752})

### Fund quality

In [28]:
good_ids = cashflows[cashflows['FundQuality'] == 'Good']['FundID'].unique()
bad_ids = cashflows[cashflows['FundQuality'] == 'Bad']['FundID'].unique()
good_ids = np.append(good_ids,bad_ids[0]) # we need at least one venture fund to make the model work
bad_ids = np.append(bad_ids,good_ids[0]) # we need at least one buyout fund to make the model work

In [29]:
port_cfs_good = portfolio_cashflows_by_id(cashflows, good_ids)
port_cfs_good = port_cfs_good.copy()    
port_cfs_bad = portfolio_cashflows_by_id(cashflows, bad_ids)
port_cfs_bad = port_cfs_bad.copy()

In [30]:
model_good = model_builder(cashflows, port_cfs_good)
model_good

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 3.451812770757449,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': 0.020473157671939912,
  'Strategy Alpha KS-PME': 0.012424508361430053,
  'Geography Alpha KS-PME': 0.01860259152407373,
  'Sizing Alpha KS-PME': 0.052154791581022764,
  'Residual Alpha KS-PME': 1.4452666373811058},
 {'Portfolio Direct Alpha': 0.07298590665988784,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': 0.000915465659180735,
  'Strategy Direct Alpha': 0.0004996393808056471,
  'Geography Direct Alpha': 0.00044625987378965615,
  'Sizing Direct Alpha': 0.0013308105021786615,
  'Residual Direct Alpha': 0.035797656065220854})

In [31]:
model_bad = model_builder(cashflows, port_cfs_bad)
model_bad

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 0.6975948123907209,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.043728953046344055,
  'Strategy Alpha KS-PME': -0.013335831330156589,
  'Geography Alpha KS-PME': 0.009612384696747522,
  'Sizing Alpha KS-PME': 0.003012157639794344,
  'Residual Alpha KS-PME': -1.1608560298071968},
 {'Portfolio Direct Alpha': -0.02022971327544504,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -0.0010429895212099444,
  'Strategy Direct Alpha': -0.0005341159045323673,
  'Geography Direct Alpha': 0.0003543075843774751,
  'Sizing Direct Alpha': 0.00030531716048397967,
  'Residual Direct Alpha': -0.05330830777327648})

### Vintage effect

In [32]:
vintage_ids = {}
for vint in cashflows['VintageYear'].unique():
    vint_ids = cashflows[cashflows['VintageYear'] == vint]['FundID'].unique()
    vintage_ids[vint] = vint_ids

In [33]:
import random
def pick_one_per_vintage(fund_dict):
    selected_ids = []

    for vintage, fund_ids in fund_dict.items():
        # Randomly pick one fund from this vintage
        fund_ids = list(fund_ids)  # ensure it's a list
        selected_id = random.choice(fund_ids)
        selected_ids.append(selected_id)

    return np.array(selected_ids)

one_from_each = pick_one_per_vintage(vintage_ids)

In [34]:
good_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 1]['FundID'].unique()
bad_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 0]['FundID'].unique()
good_vintage_ids = np.append(good_vintage_ids,one_from_each) # we need at least one venture fund to make the model work
bad_vintage_ids = np.append(bad_vintage_ids,one_from_each) # we need at least one buyout fund to make the model work

In [35]:
port_cfs_good_vintage = portfolio_cashflows_by_id(cashflows, good_vintage_ids)
port_cfs_good_vintage = port_cfs_good_vintage.copy()
port_cfs_bad_vintage = portfolio_cashflows_by_id(cashflows, bad_vintage_ids)
port_cfs_bad_vintage = port_cfs_bad_vintage.copy()

In [36]:
model_good_vintage = model_builder(cashflows, port_cfs_good_vintage)
model_good_vintage

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 2.5088879401277935,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': 0.5924528834265974,
  'Strategy Alpha KS-PME': 0.11904050250056297,
  'Geography Alpha KS-PME': 0.2084866513278696,
  'Sizing Alpha KS-PME': 0.03117677841229982,
  'Residual Alpha KS-PME': -0.3451599597774129},
 {'Portfolio Direct Alpha': 0.05454304299704395,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': 0.02001194452895142,
  'Strategy Direct Alpha': 0.003739978716301419,
  'Geography Direct Alpha': 0.005815162690374902,
  'Sizing Direct Alpha': 0.000620005314539214,
  'Residual Direct Alpha': -0.009640123431835297})

In [37]:
model_bad_vintage = model_builder(cashflows, port_cfs_bad_vintage)
model_bad_vintage

[*********************100%***********************]  1 of 1 completed


({'Portfolio KS-PME': 1.2149075485006724,
  'Market KS-PME': 1.9028910842378766,
  'Timing Alpha KS-PME': -0.6783304735773064,
  'Strategy Alpha KS-PME': 0.06540330973432007,
  'Geography Alpha KS-PME': -0.4247728334480356,
  'Sizing Alpha KS-PME': -0.011826733746159501,
  'Residual Alpha KS-PME': 0.36154319529997725},
 {'Portfolio Direct Alpha': 0.011679445611746978,
  'Market Direct Alpha': 0.03399607517871229,
  'Timing Direct Alpha': -0.02189769165668632,
  'Strategy Direct Alpha': 0.0020673508093529344,
  'Geography Direct Alpha': -0.013044646644867548,
  'Sizing Direct Alpha': -0.0005224419346110728,
  'Residual Direct Alpha': 0.011080799859846696})